<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Download-Wiki" data-toc-modified-id="Download-Wiki-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download Wiki</a></span></li><li><span><a href="#Process-and-Clean" data-toc-modified-id="Process-and-Clean-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Process and Clean</a></span></li><li><span><a href="#Generate-and-Upload" data-toc-modified-id="Generate-and-Upload-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generate and Upload</a></span></li></ul></div>

**Scribe Autosuggest Generation**

This notebook is used to run the functions found in Scribe-Data to extract, clean and load autosuggestion files into Scribe apps.

In [1]:
import os
import sys
import json
import warnings

warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
pwd = os.path.dirname(os.path.realpath("gen_autosuggestions.ipynb"))
pwd = pwd.split("scribe_data")[0]
sys.path.append(pwd)

In [3]:
from scribe_data.extract_transform.extract_wiki import download_wiki, parse_to_ndjson
from scribe_data.extract_transform.process_wiki import clean, gen_autosuggestions

# Download Wiki

In [4]:
files = download_wiki(
    language="de", target_dir="./dewiki_dump", file_limit=1, dump_id=False
)
len(files)

Files already available in the ./dewiki_dump directory.


1

In [5]:
parse_to_ndjson(
    output_path="./dewiki.ndjson",
    input_dir="./dewiki_dump",
    partitions_dir="./dewiki_partitions",
    article_limit=50,
    delete_parsed_files=True,
    multicore=True,
    verbose=True,
)

File ./dewiki.ndjson with Wikipedia articles already exists


# Process and Clean

In [6]:
with open("./dewiki.ndjson", "r") as fin:
    articles = [json.loads(l) for l in fin]

print(f"Found a total of {len(articles)} articles.")

Found a total of 50 articles.


In [7]:
article_texts = [a[1] for a in articles]

In [8]:
text_corpus = clean(
    texts=article_texts,
    ignore_words=None,
    sample_size=1,
    verbose=True,
)

Articles cleaned:   0%|          | 0/50 [00:00<?, ?article/s]

# Generate and Upload

In [9]:
autosuggest_dict = gen_autosuggestions(
    text_corpus, language="German", num_words=5, update_scribe_apps=False
)

Querying profanities to remove from suggestions.
Queried 20 words to be removed from autosuggest options.


In [10]:
autosuggest_dict

{'und': ['die', 'der', 'zur'],
 'der': ['EU', 'er', 'Europäischen'],
 'die': ['Berufsausbildung', 'er', 'sich'],
 'für': ['die', 'den', 'das'],
 'oder': ['Allgemeine', 'Zeitschrift', 'die']}